## Выгрузки по проектам

In [35]:
import xlsxwriter
import openpyxl
import pandas as pd
import numpy as np
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import openpyxl
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment

name = input('\n'+ 'Введите имя файла (выгрузки): ')
name = name + '.xlsx'
def parse_date(date_string):
    return pd.to_datetime(date_string, format='%d.%m.%Y %H:%M')

df = pd.read_excel(name, converters={'Дата окончания работы': parse_date,
                                                  'Дата начала работы': parse_date})


Введите имя файла (выгрузки): PMI_ToiMart_Февраль_2023_11472_eR9JjdL


In [36]:
if 'Выберите анкету : До работы' in df.columns:
    before_col = 'Выберите анкету : До работы'
elif 'Анкета : До работы' in df.columns:
    before_col = 'Анкета : До работы'
elif 'Выберите Анкету : До Работы' in df.columns:
    before_col = 'Выберите Анкету : До Работы'
elif 'Выберите Анкету : До работы' in df.columns:
    before_col = 'Выберите Анкету : До работы'
elif 'Выберите aнкету : До Работы' in df.columns:
    before_col = 'Выберите aнкету : До Работы'
elif 'выберите анкету : до работы' in df.columns:
    before_col = 'выберите анкету : до работы'
elif 'Выберите анкету : До Работы' in df.columns:
    before_col = 'Выберите анкету : До Работы'
elif 'выберите анкету : до работы' in df.columns:
    before_col = 'выберите анкету : до работы'
else:
    print('\n'+"Столбец с именем 'Анкета : До работы' или 'Выберите анкету : До работы' не найден!"+'\n')
    before_col = input('Введите имя столбца с информацией о том, что анкета до работы: ')

In [37]:
df['Исполнитель'] = df['Исполнитель'].apply(lambda x: x.replace('@contineo.kz', ''))

df['Дата'] = df['Дата начала работы'].dt.date
df['Дата'] = pd.to_datetime(df['Дата'])
                                                      
df = df.drop_duplicates(subset=['ID анкеты'])

df['Тип анкеты'] = np.where(df[before_col] == 1, 'До', 'После')

# group the data by 'Point: Code' and 'Work start date'
grouped = df.groupby(['Точка: Код', 'Дата'])

# count the number of 'Before' and 'After' questionnaires for each group
counts = grouped['Тип анкеты'].value_counts()

# create an empty 'Check' column
df['Проверка'] = ''

# create a DataFrame with the total counts for each day and point
total_counts = df.groupby(['Точка: Код', 'Дата']).size().reset_index(name='Total')

for index, row in total_counts.iterrows():
    point_code = row['Точка: Код']
    work_date = row['Дата']
    total_count = row['Total']
    before_count = counts[point_code, work_date, 'До'] if (point_code, work_date, 'До') in counts else 0
    after_count = counts[point_code, work_date, 'После'] if (point_code, work_date, 'После') in counts else 0
    if total_count == 2 and before_count == 1 and after_count == 1:
        df.loc[(df['Точка: Код'] == point_code) & (df['Дата'] == work_date), 'Проверка'] = 'Правильно'
    elif total_count == 2 and before_count == 2:
        df.loc[(df['Точка: Код'] == point_code) & (df['Дата'] == work_date), 'Проверка'] = 'Анкет "До" больше одной, не хватает "После"'
    elif total_count == 2 and after_count == 2:
        df.loc[(df['Точка: Код'] == point_code) & (df['Дата'] == work_date), 'Проверка'] = 'Анкет "После" больше одной, не хватает "До"'    
    elif total_count < 2:
        if before_count == 0:
            df.loc[(df['Точка: Код'] == point_code) & (df['Дата'] == work_date), 'Проверка'] = 'Не хватает анкеты "До"'
        elif after_count == 0:
            df.loc[(df['Точка: Код'] == point_code) & (df['Дата'] == work_date), 'Проверка'] = 'Не хватает анкеты "После"'
        elif before_count == 1:
            df.loc[(df['Точка: Код'] == point_code) & (df['Дата'] == work_date), 'Проверка'] = 'Одна анкета "До"'
        elif after_count == 1:
            df.loc[(df['Точка: Код'] == point_code) & (df['Дата'] == work_date), 'Проверка'] = 'Одна анкета "После"'
        else:
            print(total_count, before_count, after_count)
    elif total_count > 2:
        if before_count > 1:
            df.loc[(df['Точка: Код'] == point_code) & (df['Дата'] == work_date), 'Проверка'] = 'Анкет "До" больше одной'
        elif after_count > 1:
            df.loc[(df['Точка: Код'] == point_code) & (df['Дата'] == work_date), 'Проверка'] = 'Анкет "После" больше одной'

In [38]:
type_col = df.pop("Тип анкеты")
check_col = df.pop("Проверка")
date_col = df.pop("Дата")

df.insert(15, "Дата сохранения анкеты", df.pop("Дата сохранения анкеты"))
df.insert(15, "Дата окончания работы", df.pop("Дата окончания работы"))
df.insert(15, "Дата начала работы", df.pop("Дата начала работы"))

df.insert(15, "Дата сохранения анкеты", df.pop("Дата сохранения анкеты"))
df.insert(16, "Тип анкеты", type_col)
df.insert(17, "Проверка", check_col)
df.insert(18, "Дата", date_col)

df = df.sort_values(['Дата', 'Точка: Customer Name', 'Тип анкеты'])

with pd.ExcelWriter('new_format_'+name, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Данные', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Данные']
    worksheet.column_dimensions['A'].width = 30

### Список исполнителей, которые совершают ошибки

In [39]:
# read the Excel file with the processed data
df = pd.read_excel('new_format_' + name)

# create a new sheet in the Excel file
writer = pd.ExcelWriter('new_format_' + name, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Данные', index=False)

# group the data by 'Исполнитель', 'Point: Code', 'Дата' and 'Проверка'
grouped = df.groupby(['Исполнитель', 'Точка: Код', 'Дата', 'Проверка']).size().reset_index(name='Count')

# select the desired columns
result = grouped[['Исполнитель', 'Точка: Код', 'Дата', 'Проверка']]

result = result.drop_duplicates(subset=['Исполнитель', 'Точка: Код', 'Дата', 'Проверка'])
result = result.loc[result['Проверка'] != "Правильно"]

# sort the data by 'Дата'
result = result.sort_values('Дата')

# write the result to a new sheet in the Excel file
result.to_excel(writer, sheet_name='Проверка', index=False)

# save and close the Excel file
writer.save()

### Оформление таблицы

In [40]:
import openpyxl
from openpyxl.styles import Border, Side, Alignment, Font

# Load the Excel file
workbook = openpyxl.load_workbook('new_format_' + name)

# Select the sheet you want to format
sheet1 = workbook['Данные']
sheet2 = workbook['Проверка']

# Format sheet1
border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
for row in sheet1.iter_rows(min_row=1, max_row=sheet1.max_row, min_col=1, max_col=sheet1.max_column):
    for cell in row:
        cell.border = border

# Format sheet2
column_widths = [18] * 5
font = Font(bold=True)
for col in range(1, sheet2.max_column+1):
    sheet2.cell(row=1, column=col).font = font
    sheet2.column_dimensions[openpyxl.utils.get_column_letter(col)].width = column_widths[col-1]
sheet2.row_dimensions[1].height = 36
for row in sheet2.iter_rows(min_row=1, max_row=sheet2.max_row, min_col=1, max_col=sheet2.max_column):
    for cell in row:
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        cell.border = border

# Save the updated Excel file
workbook.save('new_format_' + name)

print('\n'+'new_format_' + name)


new_format_PMI_ToiMart_Февраль_2023_11472_eR9JjdL.xlsx


In [41]:
df['Проверка'].value_counts()

Правильно                                      524
Анкет "До" больше одной                         25
Анкет "После" больше одной                       9
Не хватает анкеты "После"                        2
Не хватает анкеты "До"                           2
Анкет "До" больше одной, не хватает "После"      2
Name: Проверка, dtype: int64